In [1]:
from scipy.stats import norm
from scipy.stats import binom
import numpy as np
import pandas as pd
import math
%load_ext cython

In [2]:
# ------------------------------------------------------------------------------
# JOHNSON DISTRIBUTIONS
# ------------------------------------------------------------------------------

# We finally decided to program our own CDF and inverse-CDF for the Johnson
# distribution functions.


# *** BASED ON PHILIPPE'S SCILAB CODE ***


# CDF for Johnson distributions
def pJohn(q, type_, a, b, c, d):
    q = np.array(q)  # converts q to an array for more efficient computations
    if type_ == "U":
    # UNBOUNDED CASE
        p = norm.cdf(a + b * np.arcsinh((q - c)/d))   # arcsin elementwise

    if type_ == "B":
    # BOUNDED CASE
        upper = c + d
        p = np.zeros(np.size(q))  # initialise with zeros
        i = np.logical_and(c < q, q < upper)  # The result is an array
        
        if np.any(i):
            # We enter here only if there is some value in `q` between
            # `c` and `upper`.
            qi = q[i]  # Just to be faster
            p[i] = norm.cdf(a + b * np.log((qi - c) / (upper - qi)))

        p[q >= upper] = 1
    return p


# Inverse-CDF for Johnson distributions
def qJohn(p, type_, a, b, c, d):
    if type_ == "U":
    # UNBOUNDED CASE
        q = c + d * np.sinh((norm.ppf(p) - a) / b)
    
    if type_ == "B":
    # BOUNDED CASE
        q = c + d / (np.exp((a - norm.ppf(p)) / b) + 1)

    return q

In [3]:
J_type = ["B", "B", "U", "U", "U", "U",
          "B", "B", "U", "U", "U", "U",
          "B", "B", "U", "U", "U", "U"]

J_a = [
0., 0., 0., 0., 0., 0.,

1.74642830857010758943626229557833166151,
3.32789132381672536784153764062847739022,
-4.85598745088089370566862569594819129775,
-1.04438933697571120946340909857749695448,
-0.52977458286145969287844983446062716444,
-0.34371386717998937758370229564494168076,

3.37153689359224037661433093037670570547,
5.21933524353541219076783885100278703868,
-4.01872982057191060039104634115859799883,
-0.75701261664619041238141624389115395037,
-0.43186873681561772117311342224111378956,
-0.29867866225722724403897426961140969985]

J_b = [
0.64646005097083720877580377544491249367,
1.39833730706480665525203763381572201528,
100.,
2.32115548141591265489566269241805350277,
1.61043109802798400472117114957805882116,
1.34925107124421982672157193726909725955,

0.69076307557374844062848335460718608085,
1.22701925790980838307542721643016935644,
1.80444211955137355943514847493631987497,
1.43196957533586078620611350537732178049,
1.20933055279040199782191033417917696741,
1.08916837329898641023346423021464511577,

0.74593166247571219250998719250234579684,
0.98133598238435636274623736299637443869,
1.08644119278200855055763353598608437671,
0.98743930478592712145401232678242473738,
0.9079732861493920122301339214072614797,
0.8555768706280539154424246218382444065]

J_c = [
-1.81531640559448629853130518464898084325,
-3.10974220782783891848630844309862248348,
0., 0., 0., 0.,

-0.48931893698288401224217306196432734367,
-1.00163621345669029289984284967824485705,
-1.41900498170269088110396585332059656851,
-0.65538265907067714949198864107359538936,
-0.3315419153227644307916851724891935819,
-0.20230241986439714646030995430481162286,

-0.27093613236330658835219937054263483016,
-0.47315502136902041809145973325494766811,
-0.5665244523471390467804271447527989781,
-0.32032500617458316656917852134863102249,
-0.18537971196280261135855039510532213063,
-0.12122093889436368817120668700925957169]

J_d = [3.63063281118897259706261036929801914047,
6.21948441565567783697261688619731113724,
100.,
2.10938136494640961313186053630477642153,
1.31177712285895824442559128408848228655,
1.,

6.6213052042202285009336108971666295627,
16.08828514862974619284545066659958256682,
0.19331807239056496916824641865444966886,
0.82361491419897282190401110495433271155,
0.73314420124628385662970998863888855497,
0.63054262902988914147483139115470665,

25.15004224768683915100766195782836377563,
97.04328850983479801334754766131519562171,
0.02805859417661557430944067988854233275,
0.3795419133008987804241261135623853673,
0.3754308073205232238191875432701887019,
0.34028822968081050623748938666543961555]

J_skewness = [0., 0., 0., 0., 0., 0.,
               2., 2., 2., 2., 2., 2.,
               5., 5., 5., 5., 5., 5.]

J_kurtosis = [-1.2, -0.6, 0.0, 1.0, 3.0, 6.0,
               4.3, 6.1, 7.9, 10.8, 16.7, 25.5,
               39.9, 52.6, 65.3, 86.4, 128.7, 192.1]

In [5]:
# STUDY 1 - Second approach (26.04.2019) - Modified for LCL <= UCL (17.10.2019)
#         - ONE-SIDED VERSION (09.01.2020)
# ------------------------------------------------------------------------------


# New values of a, b, c, d when the st. dev. of J is shifted by `tau`
# ------------------------------------------------------------------------------

# We mean the new J has:
# - the same MEDIAN
# - the same SKEWNESS
# - the same KURTOSIS
# - sigma.1 = tau*sigma.0

# THIS WORKS BECAUSE THE ORIGINAL MEDIAN OF ALL 18 `J` IS ZERO.
# If the original median was <> 0, then the new value of `c` would not be
# tau*c. The rest would remain the same, apparently.

# The old name was `new.J.sd.change`.

def shifted_J_sd(tau, type_, a, b, c, d):
    new_a = a
    new_b = b
    new_c = tau * c
    new_d = tau * d
    return new_a, new_b, new_c, new_d


# Main procedure to obtain the optimal values for p.0 and LCL/UCL, given the
# values of n, alpha.max, tau and J.id.
# ------------------------------------------------------------------------------

def min_beta_INC_onesided(tau, alpha_max, J_id, n):
    
    # Properties of the Johnson distribution being considered:
    type_ = J_type[J_id]
    a = J_a[J_id]
    b = J_b[J_id]
    c = J_c[J_id]
    d = J_d[J_id]
    
    # Parameters for the shifted J distribution:   
    a_new = 0
    b_new = 0
    c_new = 0
    d_new = 0
    
    # Parameters of J' (shifted J):
    
    # requires 'gsubfn' package
    list_ = shifted_J_sd(tau, type_, a, b, c, d)
    
    # Initialisation of the search
    best_beta = 10
    best_p0   = -1
    best_L    = -1
    actual_alpha = -1
    actual_p1 = -1
    
    # For now, we only consider these values for p.0:
    test_p0   = [0.05, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.95]
    
    for p_0 in test_p0:
        # We calculate the limits of the 'centre' of the current J distrib.:
        I_lower = qJohn(p_0/2,     type_, a, b, c, d)
        I_upper = qJohn(1 - p_0/2, type_, a, b, c, d)
        
        # We calculate p.1 the prob. of J' being OUTSIDE [I.lower, I.upper]:
        p_1 = 1 - pJohn(I_upper, type_, a_new, b_new, c_new, d_new) + pJohn(I_lower, type_, a_new, b_new, c_new, d_new)
        
        # Calculate all the probabilities we are going to need:
        binom_0 = [binom.pmf(k, n, p_0) for k in range(n+1)]  # n+1 values, indexed from 0 to n
        binom_1 = [binom.pmf(k, n, p_1) for k in range(n+1)]  # n+1 values, indexed from 0 to n
        
        ########################################################################
        # So, if we want to calculate  Pr(U == u | p.0),                       #
        # we take binom.0[(u+n)/2 + 1]  <---- Mind the `1`!!!                  #
        ########################################################################
        
        # This loop is OK for either LCL or UCL: 
        for L in range(n, -n, -2):
            # `L` stands for either `UCL` or `-LCL`  <-- Mind the `-`!!!
            if L == n:
                alpha = 0  # Will store the false alarm rate
                beta  = 1  # Will store the miss rate
             #end if
            else:
                if tau < 1:
                    alpha = alpha + binom_0[((-L - 2) + n)/2 ]
                else:
                    alpha = alpha + binom_0[((L + 2) + n)/2 ]
                
                if alpha > alpha_max:
                    break # DON'T CONTINUE WITH THIS `L` and lower!
                
                if tau < 1:
                    beta = beta - binom_1[((-L - 2) + n)/2 ]
                else:
                    beta = beta - binom_1[((L + 2) + n)/2 ]
                
                # NOTE: The function `ifelse` (see later) could have been used
                # for the two previous `if-else` sentences, but we did it like
                # that for the sake of clarity.
                
            #end else
            
            # THE CURRENT VALUES OF `p.0` and `L` are a FEASIBLE SOLUTION
            if beta < best_beta:
                # NEW IMPROVEMENT!
                best_p0 = p_0
                if tau < 1:
                    best_L = -L
                else:
                    best_L = L
                best_beta = beta
                actual_alpha = alpha
                actual_p1 = p_1
   
    # RETURNS `L`, which is `UCL` if `tau > 1` or `LCL` if `tau < 1`.
    
    return best_p0, best_L, best_beta, actual_alpha, actual_p1

In [6]:
def matrix_Q(n, p, gamma_U, gamma_Y, LCL_Y = None, UCL_Y = None):
    if LCL_Y is None:
        LCL_Y = -n
    
    if UCL_Y is None:
        UCL_Y = n
    
    # message("LCL_Y = ", LCL_Y, "; UCL_Y = ", UCL_Y)
    
    # Calculate the lower and upper limits for the value of `B_t`:
    b_min = -gamma_U + gamma_Y * (LCL_Y - 1) + 1
    b_max =  gamma_U + gamma_Y * (UCL_Y + 1) - 1    
    dimension = b_max - b_min + 1
    
    # Initalise with an empty squared matrix with as many rows and columns as
    # `dimension`:
    
    Q = np.zeros((dimension, dimension))
    
    binom_prob = [binom.pmf(i, n, p) for i in range(n)] # store all the possible values to not execute the funtion in the loop
    
    Q = loop_matrix_Q(n, p, gamma_U, gamma_Y, LCL_Y, UCL_Y, b_min, b_max, binom_prob, Q)
     
    return Q

In [7]:
%%cython

cpdef loop_matrix_Q(n, p, gamma_U, gamma_Y, LCL_Y, UCL_Y, b_min, b_max, binom_prob, Q):

    for i in range(b_min, b_max):        
            # `i` stands for the current `B_{t-1}`.

            for u in range(-n, n, 2):            
                # `u` stands for the current `U_t`.            
                y = (gamma_U*u + i) // (gamma_U + gamma_Y)

                # CHECKPOINT

                # if((y < -n) || (y > n))
                #     message("Something strange happened!")

                # `trunc` represents the rounding-towards-zero operation.
                # `y` stands for the current `Y_t`.            
                if LCL_Y <= y and y <= UCL_Y:
                    r = (gamma_U * u + i) - (gamma_U + gamma_Y) * y

                    # `r` stands for the current `R_t`.            
                    j = gamma_Y * y + r

                    # `j` is the current `B_t`.

                    # Update the position `(i,j)` in `Q`.

                    # We have to 'translate' the values `i` and `j`, which range
                    # between `b_min` and `b_max`, to values ranging from 1 to
                    # `dimension`.                
                    current_i = i - b_min
                    current_j = j - b_min

                    Q[current_i, current_j] = Q[current_i, current_j] + binom_prob[int((u + n)/2)]
    return Q

In [8]:
# Function to calculate the ARL and the SDRL of a concrete setting of our
# CEWMA chart, assuming ZERO START (thus, we need to know `p.0`).
# Valid for either the TWO-SIDED or the ONE-SIDED case.
# At least one of the two limits `LCL_Y`, `UCL_Y` should be provided.
# ------------------------------------------------------------------------------

def ARL_and_SDRL_CEWMA(n, p_0, p, gamma_U, gamma_Y, LCL_Y = None, UCL_Y = None):

    if LCL_Y is None:
        LCL_Y = -n

    if UCL_Y is None:
        UCL_Y = n

    if LCL_Y == -n and UCL_Y == n:
        #special case        
        ARL  = np.inf
        SDRL = 0

        return ARL, SDRL

    # First, we obtain the (column) vector of initial probabilities:   
    b_min = -gamma_U + gamma_Y * (LCL_Y - 1) + 1
    b_max =  gamma_U + gamma_Y * (UCL_Y + 1) - 1    
    dimension = b_max - b_min + 1

    # `initial.value` stands for `B_0`.    
    initial_value = gamma_Y * math.trunc(n * (2 * p_0 - 1)) + 0

    # `initial.value` should be between `b_min` and `b_max`. If it is not, then
    # the process is already out of control before starting!    
    if initial_value < b_min or initial_value > b_max:
        #special case        
        ARL  = 0
        SDRL = 0

        return ARL, SDRL


    # `initial.value` is between `b_min` and `b_max`:
    initial_i = initial_value - b_min

    # Vector of initial probabilities:
    q = np.zeros((dimension, 1))
    q[initial_i] = 1

    # Now, we calculate the matrix of transient probabilities, `Q`:
    Q = matrix_Q(n, p, gamma_U, gamma_Y, LCL_Y, UCL_Y)  

    # Identity matrix:    
    I = np.identity(dimension)

    # Ones vector:    
    ones = np.ones((dimension, 1))

    ARL, SDRL = matmul_solve(I, Q, q, ones)

    return ARL, SDRL

In [9]:
%%cython
import numpy as np

cpdef matmul_solve(I, Q, q, ones):
    
    aux_2 = np.linalg.solve(np.transpose((I - Q)), q)
    
    ARL = np.matmul(np.transpose(aux_2), ones)[0,0] 
    
    SDRL = (np.sqrt(2 * np.matmul(np.transpose(aux_2), np.linalg.solve((I - Q), np.matmul(Q, ones))) - ARL**2 + ARL))[0,0]
    
    return ARL, SDRL

In [10]:
def fibosearch_onesided(n, p_0, gamma_U, gamma_Y, ARL_0_min, tau):
    
    # We apply a Fibonacci search in order to find the value of `L` that has
    # to be returned.
    
    # Based on https://www.geeksforgeeks.org/fibonacci-search/.
    
    # Number of elements in the array `(-n, ..., n)`: 
    N = 2 * n + 1
    candidate_L      = None
    candidate_ARL_0  = None
    candidate_SDRL_0 = None
    ARL_0_for_n = None
    SDRL_0_for_n = None
    current_pos = None

    # Initialisation
    f_1 = 0          # (m-2)-th Fibonacci number
    f_2 = 1          # (m-1)-th Fibonacci number
    f_3 = f_1 + f_2  #     m-th Fibonacci number

    # `f_3` becomes the smallest Fibo. no. greater than or equal to `N`:

    while f_3 < N:
        f_1 = f_2
        f_2 = f_3
        f_3 = f_1 + f_2

    pos_start = 0  # offset

    #continue = ifelse(f_2 > 0, TRUE, FALSE) # NOT NECESSARY
    # list[candidate_ARL_0, candidate_SDRL_0] = PROVA(N)
    if tau < 1:
        ARL_0_for_n, SDRL_0_for_n = ARL_and_SDRL_CEWMA(n, p_0, p_0, gamma_U, gamma_Y, LCL_Y = -n)
    else:    
        ARL_0_for_n, SDRL_0_for_n = ARL_and_SDRL_CEWMA(n, p_0, p_0, gamma_U, gamma_Y, UCL_Y = n)


    # If the ARL is not feasible for `L = n`, we know that it is
    # unfeasible for all `L`, so we skip the search.
    cont = not (ARL_0_for_n < ARL_0_min)

    while cont:  # While the search is not still finished.

    # "while(f_3 > 1)" leaved us with the previous position.
    # Instead, "while(f_2 > 0)" allows us to ENTER in the loop
    # to calculate the final, right candidate ARL.0.

    # IN EACH ITERATION, WE ARE SEARCHING BETWEEN POSITIONS
    # `pos_start + 1` AND `min(pos_start + f_3, N)`
    # of the array of ARLs that has `N` positions, ranging from 1 to `N`.

        # The value returned from that interval is always the one in position
        # `pos_start + f_1` (unless this is greater than `N`).

        # (f_3 == 1) didn't work well

        # message("(", min(pos_start + f_3, N) - (pos_start + 1), ")")
        if min(pos_start + f_3, N) - (pos_start + 1) <= 0:
            cont = False # LAST ITERATION!
            # message("LAST ITERATION!")


        # message("Searching from positions ", pos_start + 1, 
        #        " to ", min(pos_start + f_3, N), ":")
        current_pos = min(pos_start + f_1, N) if f_1 != 0 else min(pos_start + 1, N)

        candidate_L = current_pos - n - 1  # Ranges from `-n` to `n`.

        # message("--- Current f_3 = ", f_3, "; current_pos = ", current_pos)
        # message("    ")

        if candidate_L == n:
            # We already had this case calculated
            candidate_ARL_0  = ARL_0_for_n
            candidate_SDRL_0 = SDRL_0_for_n
        elif tau < 1:
            candidate_ARL_0, candidate_SDRL_0 = ARL_and_SDRL_CEWMA(n, p_0, p_0, gamma_U, gamma_Y, LCL_Y = -candidate_L)
        else:    
            candidate_ARL_0, candidate_SDRL_0 = ARL_and_SDRL_CEWMA(n, p_0, p_0, gamma_U, gamma_Y, UCL_Y = candidate_L)


        # list[candidate_ARL_0, candidate_SDRL_0] = PROVA(current_pos)

        if candidate_ARL_0 < ARL_0_min:
            # The current `L` (`candidate_L`) is a NON-FEASIBLE solution.
            # We rule out all values below `L`.
            f_3 = f_2
            f_2 = f_1
            f_1 = f_3 - f_2
            pos_start = current_pos # Here we rule out all values
                                    # at or below `candidate_L`.
        else:
            # The current `L` is a FEASIBLE solution.
            # We rule out all values above the current `L`.

            f_3 = f_1        # Here we rule out values greater than `candidate_L`.
            f_2 = f_2 - f_1
            f_1 = f_3 - f_2
            # In this case, `pos_start` remains unchanged.

    # message("When we leave the loop, we have: ")
    # message("f_3 = ", f_3, "; f_2 = ", f_2, "; f_1 = ", f_1)
    # message("pos_start = ", pos_start, "; current_pos = ", current_pos)

    # We get here when `f_3` becomes 1 (and then `f_2 = 1` and `f_1 = 0`).

    return candidate_L, candidate_ARL_0

In [11]:
# Routine that returns the optimal value of the CEWMA chart parameters
# `gamma_U`, `gamma_Y`, `LCL_Y` / `UCL_Y`.
# Implements a discrete version of the golden-section search.
# ------------------------------------------------------------------------------

def optim_CEWMA_onesided(tau, ARL_0_min, J_id, n):
    
    # Properties of the Johnson distribution being considered:
    type_ = J_type[J_id]
    a = J_a[J_id]
    b = J_b[J_id]
    c = J_c[J_id]
    d = J_d[J_id]

    # Parameters for the shifted J distribution:

    a_new = 0
    b_new = 0
    c_new = 0
    d_new = 0

    GAMMA_MAX = 25 # other possible values = 100, etc.

    L_max = 2 * n + 1  # Number of possible values for `LCL_Y` and `UCL_Y`

    # Initialisation
    ARL_0   = 0
    SDRL_0  = 0
    ARL_1   = 0
    SDRL_1  = 0    
    best_ARL_1   = np.inf
    best_p_0     = None
    best_L_Y     = None
    actual_ARL_0 = None
    actual_p_1   = None

    # Parameters of J' (shifted J) (requires 'gsubfn' package):
    a_new, b_new, c_new, d_new = shifted_J_sd(tau, type_, a, b, c, d)

    # For now, we only consider these values for p_0:
    test_p0   = [0.05, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.95]

    for p_0 in test_p0:
        print("p_0 = ", p_0)
        # We calculate the limits of the 'centre' of the current J distrib.:
        I_lower = qJohn(p_0 / 2,     type_, a, b, c, d)
        I_Upper = qJohn(1 - p_0 / 2, type_, a, b, c, d)

        # We calculate `p_1`, the prob. of J' being OUTSIDE [I.lower, I_Upper]:
        p_1 = 1 - pJohn(I_Upper, type_, a_new, b_new, c_new, d_new) + pJohn(I_lower, type_, a_new, b_new, c_new, d_new)

        for gamma_U in range(1, GAMMA_MAX):
            #print("Gamma_U = ", gamma_U)

            for gamma_Y in range(1, GAMMA_MAX):

             #   if gamma_Y / 10 == math.trunc(gamma_Y/10):
              #      print(". GAMMA_Y = ", gamma_Y)
               # else:
                #    print(".")

                candidate_L = None
                candidate_ARL_0 = None

                candidate_L, candidate_ARL_0 = fibosearch_onesided(n, p_0, gamma_U, gamma_Y, ARL_0_min, tau) # <---------

                if candidate_L != None:
                    if tau < 1:
                        ARL_1, SDRL_1 = ARL_and_SDRL_CEWMA(n, p_0, p_1, gamma_U, gamma_Y, LCL_Y = -candidate_L)
                    else:
                        ARL_1, SDRL_1 = ARL_and_SDRL_CEWMA(n, p_0, p_1, gamma_U, gamma_Y, UCL_Y = candidate_L)

                    # message("He calculado ARL_1 y me da ", ARL_1)
                    if ARL_1 < best_ARL_1:  # NEW INCUMBENT!
                        print("New improvement. ARL_1 is now ", ARL_1)
                        best_ARL_1   = ARL_1
                        best_p_0     = p_0
                        best_gamma_U = gamma_U
                        best_gamma_Y = gamma_Y
                        best_L_Y   = -candidate_L if tau < 1 else candidate_L
                        actual_ARL_0 = candidate_ARL_0
                        actual_p_1   = p_1

    return best_p_0, best_gamma_U, best_gamma_Y, best_L_Y, best_ARL_1, actual_ARL_0, actual_p_1[0]

1. Probar matrices dispersas
2. Identificar el cuello de botella, ver solución
3. Comparar resultados con el gráfico Shewhart
4. Programar genético/heurístico para mejorar velocidad
5. Plantear la posibilidad de red neuronales
6. Simulacion Montecarlo para verificar. IN J,n,tau, po, gammas,L. Anotar la cantidad de rachas hasta fuera de control. 

In [33]:
# RESULTS INTO TABLE
# ------------------------------------------------------------------------------


def CEWMA_tests():
    
    # Values for each initial parameter:
    test_alpha_0 = 0.0027  # <--- Changed notation to match the paper
    test_n       = [10, 15, 20, 25, 30]
    test_tau     = [0.25, 0.5, 0.75, 1.25, 2, 4]
    test_J_id    = [i for i in range(18)]   
    ARL_0        = 1 / test_alpha_0
    
    # Start the data frame with the column names
    df = pd.DataFrame(columns=['alpha.0', 'n', 'tau', 'J.id', 'p.0', 'Gamma.U', 'Gamma.Y','L.Y', 
                               'alpha', 'ARL.0', 'p.1', 'beta', 'ARL.1', 'ARL.var', 'J.skew', 'J.kurt'])
    
    
    count = 0
    loop_size = len(test_n) * len(test_tau) * len(test_J_id)
    
    # Start of the main loop 
    for n in test_n:
        for tau in test_tau:
            for J_id in test_J_id:
                count += 1
                
                # Indicate the progress
                print(40 * '#', count, 'of', loop_size , 40 * '#')
                
                # Main function and computation time
                best_p_0, best_gamma_U, best_gamma_Y, best_L_Y, best_ARL_1, actual_ARL_0, actual_p_1 = optim_CEWMA_onesided(tau, ARL_0, J_id, n)
                
                # Aditional information for each iteration, useful to compare with same case in Shewhart
                alpha = 1 / actual_ARL_0
                beta = 1 - 1 / best_ARL_1
                ARL_var = (best_ARL_1 - actual_ARL_0) / actual_ARL_0
                J_skew  = J_skewness[J_id]
                J_kurt  = J_kurtosis[J_id]
                
                # save the iteration in a data frame row
                df2 = pd.DataFrame([[test_alpha_0, n, tau, J_id, best_p_0, best_gamma_U, best_gamma_Y, best_L_Y, alpha, 
                   actual_ARL_0, actual_p_1, beta, best_ARL_1, ARL_var, J_skew, J_kurt]], 
                   columns=['alpha.0', 'n', 'tau', 'J.id', 'p.0', 'Gamma.U', 'Gamma.Y','L.Y', 
                            'alpha', 'ARL.0', 'p.1', 'beta', 'ARL.1', 'ARL.var', 'J.skew', 'J.kurt'])

                # Add the iteration row to full data base
                df = df.append(df2, ignore_index=True)
    
                # Store the row in a CSV file inmediately, so they can be accessed and saved while the loop continues
                # if no folder details are given, it will be saved in the same directory as this .py file
                df.to_csv('CEWMA_tests.csv')





In [ ]:
CEWMA_tests()